In [24]:
# full width
from IPython.core.display import display,HTML
display(HTML("<style>{ width:100%;}</style>"))

In [4]:
%load_ext autoreload
%autoreload 2

In [25]:
import os, math, subprocess
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from IPython.display import display

# # 
# from lib_modeling import *
# from lib_feature_engineering import *

# some settings for displaying Pandas results
pd.set_option('display.width', 2000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision', 4)
pd.set_option('display.max_colwidth', -1)

In [26]:
# load train data
data_path = "home-credit-default-risk/application_train.csv"
pdf_train = pd.read_csv(data_path)

# filter by tvt code
pdf_tvt_extend = pd.read_pickle("pdf_tvt_extend.pkl", compression="bz2")
pdf_train_filtered = (pdf_tvt_extend.query("tvt_code == 'train'")
                      .merge(pdf_train[["SK_ID_CURR"]], on="SK_ID_CURR")
                      .drop(columns=["tvt_code"]))
pdf_train_filtered.head()

FileNotFoundError: [Errno 2] No such file or directory: 'pdf_tvt_extend.pkl'

In [27]:
# load bureau
data_path = "home-credit-default-risk/bureau.csv"
pdf_data = pd.read_csv(data_path)
pdf_data

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,9.1323e+04,0.0000e+00,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,2.2500e+05,1.7134e+05,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,4.6432e+05,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,9.0000e+04,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.500,0,2.7000e+06,NaN,NaN,0.0,Consumer credit,-21,NaN
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.000,0,1.8000e+05,7.1017e+04,108982.620,0.0,Credit card,-31,NaN
6,215354,5714468,Active,currency 1,-43,0,79.0,NaN,0.000,0,4.2104e+04,4.2104e+04,0.000,0.0,Consumer credit,-22,NaN
7,162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.000,0,7.6878e+04,0.0000e+00,0.000,0.0,Consumer credit,-1710,NaN
8,162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.000,0,1.0301e+05,0.0000e+00,0.000,0.0,Consumer credit,-840,NaN
9,162297,5714471,Active,currency 1,-1146,0,-484.0,NaN,0.000,0,4.5000e+03,0.0000e+00,0.000,0.0,Credit card,-690,NaN


In [28]:
print('before')
print(pdf_data['CREDIT_ACTIVE'].value_counts())
print('after')
pdf_data.loc[pdf_data['CREDIT_ACTIVE']=='Sold','CREDIT_ACTIVE'] ='Bad debt'
print(pdf_data['CREDIT_ACTIVE'].value_counts())

before
Closed      1079273
Active      630607 
Sold        6527   
Bad debt    21     
Name: CREDIT_ACTIVE, dtype: int64
after
Closed      1079273
Active      630607 
Bad debt    6548   
Name: CREDIT_ACTIVE, dtype: int64


In [29]:
# những currency hiếm gặp sẽ đưa vào chung một loại currency 2
print('before')
print(pdf_data['CREDIT_CURRENCY'].value_counts())
pdf_data.loc[pdf_data['CREDIT_CURRENCY'].isin(['currency 3','currency 4']),'CREDIT_CURRENCY'] = 'currency 2'
print(pdf_data['CREDIT_CURRENCY'].value_counts())

before
currency 1    1715020
currency 2    1224   
currency 3    174    
currency 4    10     
Name: CREDIT_CURRENCY, dtype: int64
currency 1    1715020
currency 2    1408   
Name: CREDIT_CURRENCY, dtype: int64


In [33]:
pdf_data['CREDIT_TYPE'].value_counts().to_frame('freq')

,freq
Consumer credit,1251615
Credit card,402195
Car loan,27690
Mortgage,18391
Microloan,12413
Loan for business development,1975
Another type of loan,1017
Unknown type of loan,555
Loan for working capital replenishment,469
Cash loan (non-earmarked),56


In [34]:
pdf_check_credit_type = pdf_train_filtered.merge(pdf_data[["SK_ID_CURR", "CREDIT_TYPE"]]).groupby(["CREDIT_TYPE", "TARGET"]).size().to_frame("count")
pdf_check_credit_type = pdf_check_credit_type.reset_index().set_index("CREDIT_TYPE")

# 
pdf_pct = pdf_check_credit_type.query("TARGET == 1").join(pdf_distr_credit_type)
pdf_pct["pct"] = pdf_pct["count"] * 100.0 / pdf_pct["freq"]
pdf_pct.sort_values("pct")

NameError: name 'pdf_train_filtered' is not defined

In [35]:
# dựa vào phần trăm tín dụng xấu lớn hơn 5% và có phân bố nhỏ sẽ được xếp vào good/bad loan
ls_good_loan = [
    "Mobile operator loan",
    "Interbank credit",
    "Loan for purchase of shares (margin lending)",
    "Real estate loan"
    "Cash loan (non-earmarked)",
    "Another type of loan"
]

ls_bad_loan = [
    "Loan for the purchase of equipment",
    "Microloan",
    "Loan for working capital replenishment"
]
pdf_data.loc[pdf_data['CREDIT_TYPE'].isin(ls_good_loan),'CREDIT_TYPE'] ='Consumer credit'
pdf_data.loc[pdf_data['CREDIT_TYPE'].isin(ls_bad_loan),'CREDIT_TYPE']= 'Bad loan'
print('After')
print(pdf_data['CREDIT_TYPE'].value_counts())

After
Consumer credit                  1252638
Credit card                      402195 
Car loan                         27690  
Mortgage                         18391  
Bad loan                         12901  
Loan for business development    1975   
Unknown type of loan             555    
Cash loan (non-earmarked)        56     
Real estate loan                 27     
Name: CREDIT_TYPE, dtype: int64


In [36]:
%%time
dict_feat = {
    "CREDIT_ACTIVE": ['Closed', 'Active', 'Bad debt'],
    "CREDIT_CURRENCY": ['currency 1', 'currency 2'],
    "CREDIT_TYPE": ["Consumer credit", "Credit card", "Car loan", "Mortgage", "Loan for business development", "Unknown type of loan", "Bad loan"],
}
pdf_onehot = gen_one_hot_feat(pdf_data, dict_feat)

NameError: name 'gen_one_hot_feat' is not defined